In [1]:
import numpy as np
import datetime
import torch
import torchvision
import glob
import cv2

In [5]:
class CustomDatasetFolder(torchvision.datasets.DatasetFolder):
    def __init__(self, loader, cuda_device='cpu', path="datasets_segmentation/Brain/test/", img_dim=(256, 256)):
        self.loader = loader
        self.cuda_device = cuda_device
        self.imgs_path = path
        file_list = glob.glob(self.imgs_path + "*[!_mask].npy") # All non-mask
        self.data = []
        for img in file_list:
            mask = img[:-4] + "_mask.npy"
            self.data.append([img, mask])
        self.img_dim = img_dim
        self.num_samples = len(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, img_mask_path = self.data[idx]
        img = self.loader(img_path)
        img_mask = self.loader(img_mask_path, True)
        
        return img, img_mask

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, cuda_device='cpu', path="datasets_segmentation/Brain/test/", img_dim=(256, 256)):
        self.cuda_device = cuda_device
        self.imgs_path = path
        file_list = glob.glob(self.imgs_path + "*[!_mask].npy") # All non-mask
        self.data = []
        for img in file_list:
            mask = img[:-4] + "_mask.npy"
            self.data.append([img, mask])
        self.img_dim = img_dim
        self.num_samples = len(self.data)
        
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self, idx):
        img_path, img_mask_path = self.data[idx]
        img = np.load(img_path)
        img_mask = np.load(img_mask_path)
        img_tensor = torch.from_numpy(img).to(self.cuda_device)
        img_mask_tensor = torch.from_numpy(img_mask).to(self.cuda_device)
        
        
#         img = cv2.imread(img_path)
#         img = img / 255
#         img = cv2.resize(img, self.img_dim)
#         img_mask = cv2.imread(img_mask_path)
#         img_mask = img_mask / 255
#         img_mask = cv2.resize(img_mask, self.img_dim)
#         img_tensor = torch.from_numpy(img).float().to(self.cuda_device)
#         img_tensor = img_tensor.permute(2, 0, 1)
#         img_mask_tensor = torch.from_numpy(img_mask).float().to(self.cuda_device)
#         img_mask_tensor = img_mask_tensor.permute(2, 0, 1)
        
        return img_tensor, img_mask_tensor

In [4]:
# ds = CustomDataset()

In [5]:
# data_loader = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=True)

In [7]:
class LoopLoader():
    def __init__(self,
            dset_path,
            which, # '['train', 'test', 'val']'
            batch_size,
            cuda_device
        ):

        self.dset_path = dset_path
        self.which = which
        self.batch_size = batch_size
        self.cuda_device = cuda_device

        # For a list of which, we concatenate
#         self.ds_folder = torch.utils.data.ConcatDataset([torchvision.datasets.DatasetFolder(
#             root=f"{dset_path}/{split}",
#             extensions="npy",
#             loader=gen_loader(self.cuda_device))
#             for split in which])
        
#         self.ds_folder = torch.utils.data.ConcatDataset([CustomDataset(
#             path = f"{self.dset_path}/{split}/") 
#             for split in which])

        self.ds_folder = torch.utils.data.ConcatDataset([CustomDatasetFolder(
            path=f"{dset_path}/{split}/",
            cuda_device='cpu',
            loader=segmentation_loader(self.cuda_device))
            for split in which])

        self.data_loader = torch.utils.data.DataLoader(
            dataset=self.ds_folder,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=True)

        self.reload_iter()


    def reload_iter(self):
        self.data_loader = torch.utils.data.DataLoader(
            dataset=self.ds_folder,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=True)
        self.loader_iter = iter(self.data_loader)


    def get_batch(self):
        try:
            return next(self.loader_iter)
        except StopIteration:
            self.reload_iter()
            return next(self.loader_iter)

    
def segmentation_loader(cuda_device):
    def the_loader(path, mask=False):
        # Load data
        ary = np.load(path)
        # If mask, add third channel
        if mask: ary.shape = (1, *ary.shape)
        # Send the tensor to the GPU/CPU depending on what device is available
        tensor = torch.from_numpy(ary).to(cuda_device)
        return tensor
    return the_loader


# Just putting the cuda_device in a closure for the DatasetFolder loader
def gen_loader(cuda_device):
    # Load an image, convert it to a tensor with one single channel,
    # and send it to the cuda device (GPU/CPU)
    def the_loader(path):
        # Load the data: a 2D numpy array
        ary = np.load(path)
        # We need a 3rd dimension for "channels"
        ary.shape = (1, *ary.shape) # same as `reshape()` but "inplace"
        # Send the tensor to the GPU/CPU depending on what device is available
        tensor = torch.from_numpy(ary).to(cuda_device)
        return tensor
    return the_loader

In [8]:
# ll = LoopLoader('datasets/Prostate', ['train', 'val', 'test'], 8, 'cpu')
ll = LoopLoader('datasets_segmentation/Brain', ['train', 'val', 'test'], 8, 'cpu')
b1 = ll.get_batch()

In [9]:
b1[0]

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [9]:
torch.max(b1[0])

tensor(0.9451, dtype=torch.float64)

# Work with .tif

In [21]:
from PIL import Image
im = Image.open('dsets_unet/brain_mri/TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_1.tif')
nim = np.array(im)

In [23]:
nim.shape

(256, 256, 3)

In [42]:
nim2 = np.load("datasets/Prostate/val/0/dwi-mri-8-ep2ddifftraDYNDISTCALCBVAL-22625_fid-3_visit0_augmented-6.npy")

In [37]:
nim2.shape

(65, 65)

In [47]:
pil_image=Image.fromarray(nim2 * 255)
pil_image.show()

In [45]:
nim2 * 10

array([[-5.7441244, -5.697813 , -5.7096796, ..., -1.2457886, -4.446393 ,
        -6.2011404],
       [-5.7110524, -5.712204 , -5.706325 , ...,  1.6483636, -3.3304226,
        -6.399544 ],
       [-5.710719 , -5.711265 , -5.6979265, ...,  1.2752781, -3.1896305,
        -6.243905 ],
       ...,
       [ 9.724371 ,  9.293088 ,  7.8892255, ..., -5.7109904, -5.7109904,
        -5.7109904],
       [10.4468975,  9.636963 ,  8.050376 , ..., -5.7109904, -5.7109904,
        -5.7109904],
       [10.713201 , 10.140901 , 10.107052 , ..., -5.7109904, -5.7109904,
        -5.7109904]], dtype=float32)

In [51]:
import os

path = "./dsets_unet/brain_mri"
dir_list = os.listdir(path)

In [68]:
images = []
# Iterate over all the directories
for dir_name in dir_list:
    dir_path = path + '/' + dir_name
    if os.path.isdir(dir_path):
        for image in os.listdir(dir_path):
            if not image.endswith('_mask.tif'):
                i = dir_path + '/' + image
                m = dir_path + '/' + image[:-4] + '_mask.tif'
                images.append([i, m])

In [70]:
len(images)

3929

In [74]:
# os.makedirs('./dsets_unet/J/test')

In [76]:
import torch

In [80]:
train_data, val_data, test_data = torch.utils.data.random_split(images, [3000, 900, 29])

In [83]:
import shutil

In [84]:
for data in train_data:
    im_name0 = data[0].split('/')[-1]
    im_name1 = data[1].split('/')[-1]
    shutil.copyfile(data[0], f'dsets_unet/J/train/{im_name0}')
    shutil.copyfile(data[1], f'dsets_unet/J/train/{im_name1}')
    
for data in test_data:
    im_name0 = data[0].split('/')[-1]
    im_name1 = data[1].split('/')[-1]
    shutil.copyfile(data[0], f'dsets_unet/J/test/{im_name0}')
    shutil.copyfile(data[1], f'dsets_unet/J/test/{im_name1}')
    
for data in val_data:
    im_name0 = data[0].split('/')[-1]
    im_name1 = data[1].split('/')[-1]
    shutil.copyfile(data[0], f'dsets_unet/J/val/{im_name0}')
    shutil.copyfile(data[1], f'dsets_unet/J/val/{im_name1}')